In [0]:
data = [
    (1, "South", "Mobile", 500, "2023-10-01"),
    (2, "South", "Laptop", 800, "2023-10-02"),
    (3, "North", "Mobile", None, "2023-10-03"),  
    (4, "East", "Tablet", 650, "2023-10-04"),
    (4, "East", "Tablet", 650, "2023-10-04"),     
    (5, None, "Mobile", 550, "2023-10-05")        
]
columns = ["id", "region", "product", "amount", "date"]
bronze_df=spark.createDataFrame(data, columns).repartition(1)
display(bronze_df)


In [0]:
bronze_path="/Volumes/bronze_databricks/default/bronze_practise"
bronze_df.write.mode("overwrite").option("header",True).csv(bronze_path)
display(bronze_df)

In [0]:
df_read=spark.read.csv(bronze_path,header=True)
display(df_read)



In [0]:
silver_df=df_read.dropDuplicates().fillna({'region':"unknown",'amount':0})
display(silver_df)

In [0]:
silver_path="/Volumes/bronze_databricks/default/silver_practise/silver_path"
r=silver_df.write.mode("overwrite").format("delta").save(silver_path)
display(r)
df_read=spark.read.format("delta").load(silver_path)
display(df_read)


In [0]:
from pyspark.sql.functions import sum,avg
gold_df=silver_df.groupBy("region").agg(sum("amount").alias("total_sales"),avg("amount").alias("avg_sales"))
display(gold_df)

In [0]:

f=gold_df.write.mode("overwrite").format("delta").save("/Volumes/bronze_databricks/default/gold_practise/gold_pathss")
gold_read=spark.read.format("delta").load("/Volumes/bronze_databricks/default/gold_practise/gold_pathss")
display(gold_read)

In [0]:
r=gold_df.write.mode("overwrite").format("csv").save("/Volumes/bronze_databricks/default/gold_practise")
display(r)
gold_reads=spark.read.csv("/Volumes/bronze_databricks/default/gold_practise",header=True)
display(gold_reads)